In [5]:
import pandas as pd
import numpy as np
from transformers import AutoModel, AutoTokenizer, get_cosine_schedule_with_warmup, AutoConfig
import torch
import torch.nn as nn
from torch.utils.data import Sampler, Dataset, DataLoader
from IPython.display import display
# from accelerate import Accelerator
from tqdm.notebook import tqdm
import random
import os
import multiprocessing
from sklearn.model_selection import StratifiedKFold
# import more_itertools
from sklearn.metrics import f1_score

import string
from bs4 import BeautifulSoup
import re

In [1]:
df_train=pd.read_csv(r'D:\BaiduNetdiskDownload\python\项目\文本处理项目\nlp-getting-started\train.csv')
df_test=pd.read_csv(r'D:\BaiduNetdiskDownload\python\项目\文本处理项目\nlp-getting-started\test.csv')

In [3]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7613 entries, 0 to 7612
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        7613 non-null   int64 
 1   keyword   7552 non-null   object
 2   location  5080 non-null   object
 3   text      7613 non-null   object
 4   target    7613 non-null   int64 
dtypes: int64(2), object(3)
memory usage: 297.5+ KB


In [6]:
def seed_everything(seed=42):
    random.seed(seed) #
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

seed_everything(seed=42)

In [26]:
random.seed(10)
np.random.seed(10)
for i in range(20):
    print(np.random.random())

0.771320643266746
0.0207519493594015
0.6336482349262754
0.7488038825386119
0.4985070123025904
0.22479664553084766
0.19806286475962398
0.7605307121989587
0.16911083656253545
0.08833981417401027
0.6853598183677972
0.9533933461949365
0.003948266327914451
0.5121922633857766
0.8126209616521135
0.6125260668293881
0.7217553174317995
0.29187606817063316
0.9177741225129434
0.7145757833976906


In [27]:
os.environ['PYTHONHASHSEED']=str(10)

In [30]:
%s

UsageError: Line magic function `%s` not found.


In [28]:
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [29]:
punct=string.punctuation
f'{punct}'

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [33]:
mispell_dict = {"aren't" : "are not",
"can't" : "cannot",
"couldn't" : "could not",
"couldnt" : "could not",
"didn't" : "did not"}
mispellings_re = re.compile('(%s)' % '|'.join(mispell_dict.keys()))#%s,字符串占位符
mispellings_re
mispellings_re = re.compile('|'.join(mispell_dict.keys()))
mispellings_re
def replace_typical_misspell(text):
    mispellings_re = re.compile('(%s)' % '|'.join(mispell_dict.keys()))  #将间歇性构成构建正则表达式

    def replace(match):
        print(match)#<re.Match object; span=(15, 20), match="can't">

        print(match.group(1))#can't

        print(mispell_dict[match.group(0)])
        return mispell_dict[match.group(0)]

    # print(mispellings_re.sub(replace, text))
    return mispellings_re.sub(replace, text)

re.compile(r"aren't|can't|couldn't|couldnt|didn't", re.UNICODE)

In [88]:
a="@MythGriy they can't detonate unless they touch the ground"
replace_typical_misspell(a)

<re.Match object; span=(15, 20), match="can't">
can't
cannot


'@MythGriy they cannot detonate unless they touch the ground'

In [83]:
df_train['text'][2652]

"@MythGriy they can't detonate unless they touch the ground"

In [84]:
re.sub(mispellings_re,mispell_dict[],df_train['text'][2652])

TypeError: sub() missing 1 required positional argument: 'string'

In [92]:
BeautifulSoup(a).text.strip().lower()

"@mythgriy they can't detonate unless they touch the ground"

In [93]:
re.sub(r'\s+',' ',a)#\s:用于匹配单个空格符,包括tab键和换行符; \S:用于匹配除单个空格符之外的所有字符

"@MythGriy they can't detonate unless they touch the ground"

In [94]:
re.sub(r'((http)\S+)','http',a)

"@MythGriy they can't detonate unless they touch the ground"

In [95]:
if cfg.debug:

SyntaxError: incomplete input (1951619052.py, line 1)

In [ ]:
class cfg():
    #构建参数
    max_len = 50
    model_name = "microsoft/deberta-v3-large"
    train_batch_size = 32
    reinit_layers = 2
    valid_batch_size = 64
    fold = 1
    device = "cuda" if torch.cuda.is_available() else "cpu"
    bilstm_hidden = 256
    epochs = 5
    n_folds = 5  #分几类？
    debug = False
    train_folds = [0, 1, 2, 3, 4]

In [96]:
import numpy as np
from sklearn.model_selection import StratifiedKFold
X = np.array([[1, 2], [3, 4], [1, 2], [3, 4]])
y = np.array([0, 0, 1, 1])
skf = StratifiedKFold(n_splits=2)
skf.get_n_splits(X, y)

2

In [97]:
print(skf)

StratifiedKFold(n_splits=2, random_state=None, shuffle=False)


In [ ]:
    StratifiedKFold(n_splits=2, random_state=None, shuffle=False)
    >>> for i, (train_index, test_index) in enumerate(skf.split(X, y)):
    ...     print(f"Fold {i}:")
    ...     print(f"  Train: index={train_index}")
    ...     print(f"  Test:  index={test_index}")
    Fold 0:
      Train: index=[1 3]
      Test:  index=[0 2]
    Fold 1:
      Train: index=[0 2]
      Test:  index=[1 3]

In [102]:
 import tokenize    #词法分析器
 example = tokenize(a, max_length=50,
                                 padding="max_length", add_special_tokens=True, truncation=True)
 example

TypeError: 'module' object is not callable

In [103]:
dp=[[0,1]]*(5)

In [104]:
dp

[[0, 1], [0, 1], [0, 1], [0, 1], [0, 1]]